# Evolved ape transaction data analysis

This notebook uses opensea api to download transaction data, then spotlights buyers that worth further notice. 

To run the analysis part, you may start with "analysis" section.

In [2]:
import requests
import json
import pandas as pd
import numpy as np
import time
import OpenSeaAPIKey 
import warnings
from datetime import datetime
import os
import glob
import pickle

# dir_path = os.path.dirname(os.path.realpath(__file__))
# test_path = os.path.join(dir_path, "test_request_dict.pkl")


# Get opensea data

In [3]:


warnings.filterwarnings('ignore')
CURRENT_DATE = datetime.now().strftime("%d-%m-%Y")

# pd.options.display.max_columns = None
# pd.options.display.max_colwidth = None

class OpenSeaAssets:
    

    def __init__(self, **kwargs):
        """
        Initialize the class with the following parameters:
        lower_offset: int, the lower offset to start the data retrieval
        upper_offset: int, the upper offset to end the data retrieval
        limit: int, the limit of data to be retrieved per request (max is 50)
        collection: str, slug of the collection to be retrieved
        data_folder: str, path to the folder where the data were saved
        add_to_data: bool, if True, the data will be added to the existing data
        prediction_mode: bool, if True, the data will be retrieved for prediction
        token_ids: int, the token id to be retrieved
        """
        self.lower_offset = kwargs.get('lower_offset', 0)
        self.upper_offset = kwargs.get('upper_offset', 1)
        self.limit = kwargs.get('limit', 50)
        self.collection = kwargs.get('collection', 'a')
        self.asset_contract_address = kwargs.get('asset_contract_address', 'a')
        self.data_folder = kwargs.get('data_folder', 'a')
        self.add_to_data = kwargs.get('add_to_data', False)
        self.prediction_mode = kwargs.get('prediction_mode', False)
        # To retrieve data of just one asset
        try:
            self.token_ids = kwargs.get('token_ids')
        except:
            pass


    def set_init(self, **kwargs):
        """
        Input:
        ------
        Initialize the request to retrieve the data

        Output:
        -------
        response.text: str, the response of the request
        response: response code of the request
        """
        headers = {
            "Accept": "application/json",
            "X-API-KEY": OpenSeaAPIKey.X_API_KEY
        }
        url_parameters = ''
        for key, value in kwargs.items():
            if value is not None:
                if self.prediction_mode == True:
                    if (key == 'token_ids') or (key == 'asset_contract_address'):
                        url_parameters += f'{key}={value}&'
                else:
                    url_parameters += f'{key}={value}&'
        url = f"https://api.opensea.io/api/v1/assets?{url_parameters}".rstrip('&')
        print(f"[Retrieving] url: {url}")
        response = requests.request("GET", url, headers=headers)
        return response.text, response

    def new_data_validator_(self, df, ascending):
        """
        Input:
        ------
        df: pd.DataFrame, the dataframe to be validated
        ascending: bool, if True, the dataframe will be sorted in ascending order
        
        Output:
        -------
        df: Pandas DataFrame, first row of the dataframe"""
        df = df[df.last_sale.notnull()]
        df['last_sale_date'] = df.last_sale.apply(lambda cell: cell['event_timestamp'])
        df.sort_values(by=['last_sale_date'], ascending=ascending, inplace=True)
        return df.head(1).reset_index(drop=True)
    
    def get_data(self):
        """
        Retrieve the data from the API chuck by chuck and save it to a pickle file

        Output:
        -------
        df: Pandas DataFrame, the dataframe with the data retrieved
        error_codes: dummy variable, internal use only
        """
        error_codes = {"error_points":[], "limit":[]}
        print("="*10,self.collection,"="*10)
        for offset_ in range(self.lower_offset, self.upper_offset):
            initial_parameters = {
            # 'format':'json',
            'offset':offset_*self.limit,
            'limit':self.limit,
            'asset_contract_address':self.asset_contract_address,
            'order_by':'sale_date',
            'order_direction':'asc',
            'collection_slug':self.collection,
            #'token_ids':self.token_ids,

            }
            try:
                json_, response_code = self.set_init(**initial_parameters)
                print("="*20,"retrieve_collection_data.py","="*20)
                print(f"[test] response_code: {response_code} | json_: {json_}")
                parsed_json = json.loads(json_)
                #print(f"[test] parsed_json: {parsed_json}")
                collections_info = parsed_json['assets']
                if (self.prediction_mode == True) and (len(collections_info) <= 0):
                    # return []
                    return ("df", "error_codes", collections_info)
                if len(collections_info) <= 0:
                    break
                if (initial_parameters['offset'] != self.lower_offset ) and (initial_parameters['offset'] % 900 == 0):
                    time.sleep(5)
            except:
                print(f"[Error] response: {response_code} | offset: {offset_}")
                error_codes['error_points'].append(offset_)
                error_codes['limit'].append(self.limit)
                time.sleep(15)
                
            df = pd.DataFrame(collections_info)
            if self.prediction_mode != True:
                df.to_pickle(f"{self.data_folder}/{CURRENT_DATE}_{self.collection}_assets_{offset_*self.limit}_to_{offset_*self.limit+self.limit}.pkl")
            if self.add_to_data == True:
                df_local = pd.read_pickle(f"{self.data_folder}/{self.collection}.pkl")
                df_local_head = self.new_data_validator_(df_local, False)
                df_latest_head = self.new_data_validator_(df, True)
                if (df_local_head.iloc[0]['last_sale_date'] >= df_latest_head.iloc[0]['last_sale_date']):
                    print(f"[Info] No more new data found for {self.collection}")
                    break
        return df, error_codes, collections_info


In [ ]:


## How it works ##
for slug in ['evolved-apes-inc']: 
    data_folder = slug+'_data'
    if not os.path.isdir(data_folder):
        os.makedirs(data_folder)
    init_vals = {
        "lower_offset":0,
        "upper_offset":200,
        "limit":50,
        "collection":slug,
        "data_folder":data_folder,
        "asset_contract_address":'0x233a65b06Ef83CCf2fd58832086dD782f9da1642'
        
        }
    osa = OpenSeaAssets(**init_vals)
    osa.get_data()
    files = glob.glob(f'{data_folder}/*.pkl')
    df = pd.concat([pd.read_pickle(fp) for fp in files], ignore_index=True)
    df.to_pickle(f"{data_folder}/{slug}.pkl")

In [ ]:
slug = 'evolved-apes-inc'
data_folder = slug+'_data'
df=pd.read_pickle(f"{data_folder}/{slug}.pkl")

In [ ]:
df['event_time'] = np.nan
df['to_account']=''
df['from_account']=''
df['price_eth']=np.nan
for kr,row in df.iterrows():
    if row['last_sale']['transaction'] is not None:
        df['event_time'].iloc[kr] = row['last_sale']['event_timestamp'] 
        df['from_account'].iloc[kr] = row['last_sale']['transaction']['from_account']['address']
        df['to_account'].iloc[kr] = row['last_sale']['transaction']['to_account']['address']
        lastsale = row['last_sale']
        totalethprice = float(lastsale['total_price']) *(10**-(float(lastsale['payment_token']['decimals'])))*(float(lastsale['payment_token']['eth_price']))
        df['price_eth'].iloc[kr] = totalethprice/ float(lastsale['quantity'])




In [ ]:
shortdf = df.dropna(subset=['event_time'])
shortdf = shortdf[['event_time','to_account','from_account','price_eth','token_id','permalink']]

In [ ]:
shortdf.to_csv('ape_transaction.csv')

# data analysis

In [4]:
shortdf=pd.read_csv('ape_transaction.csv')

In [5]:
shortdf = shortdf.sort_values(by='event_time')

In [8]:
import plotly.express as px
px.scatter(shortdf.iloc[:6800],x='event_time',y='price_eth',width=700)

## who buys / sells the highest price assets?

In [ ]:
price_threshold =1
highprice_df = shortdf.loc[shortdf['price_eth']>price_threshold]
highprice_buyers = set(highprice_df['from_account'].values)

In [ ]:
highprice_df.loc[highprice_df['from_account']=='0x8ed6d5a84ebf96e81d0d66bcc8216192cbf0bd2a']

for these high price nft buyers, how much do they buy and sell? do we see some patterns?

In [ ]:
buyer_dict = {'address':[],'total_buy':[],'total_buy_eth':[],'av_price_eth':[],'total_sell':[],'total_sell_eth':[],} # not sure if should count one nft trading amount >1
for buyer in highprice_buyers:
    buyer_dict['address'].append(buyer)
    buyer_dict['total_buy'].append(len(shortdf.loc[shortdf['from_account']==buyer]))
    buyer_dict['total_buy_eth'].append(sum(shortdf.loc[shortdf['from_account']==buyer]['price_eth']))
    buyer_dict['av_price_eth'].append(np.mean(shortdf.loc[shortdf['from_account']==buyer]['price_eth']))
    buyer_dict['total_sell'].append(len(shortdf.loc[shortdf['to_account']==buyer]))
    buyer_dict['total_sell_eth'].append( sum(shortdf.loc[shortdf['to_account']==buyer]['price_eth']))
    


In [9]:
shortdf.loc[shortdf['to_account']=='0x8ed6d5a84ebf96e81d0d66bcc8216192cbf0bd2a']

,Unnamed: 0,event_time,to_account,from_account,price_eth,token_id,permalink


In [10]:
shortdf.loc[shortdf['from_account']=='0x8ed6d5a84ebf96e81d0d66bcc8216192cbf0bd2a']

,Unnamed: 0,event_time,to_account,from_account,price_eth,token_id,permalink
1527,1527,2021-09-27T20:58:07,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,0x8ed6d5a84ebf96e81d0d66bcc8216192cbf0bd2a,3.0,23,https://opensea.io/assets/ethereum/0x233a65b06...


In [ ]:
df = pd.DataFrame(buyer_dict).sort_values(by='av_price_eth',ascending=False)
df.iloc[:10]

ideas:
- transaction data
    - "fake wallet" or "real wallet"? judging by what type of nft they buy/transfer
    - wallet relation to investigate wash trading
        - self-funding, i.e. wallet funded by the seller or by the funder for the seller
- centralized wallet control (who can do the rug pull): check the contract?
    - how does rug-pulling actually happen?
    - any special discovery of those wallets that received rug-pull results?
- twitter: communication frequency?
- existing tools: chainanalysis sanction list?




### top buyers: do they only buy this nft?
Ranking by the average amount they pay for an evolved-ape nft:
- 2 out of 3 top buyers only bought this nft in their whole transaction history.
- another top 5 buyer only bought this and one other nft.

### the action of rug pulling
8+6=14 wallets received funds, some are new wallets, some have previous records.
Pullling happened in an 36 hour time span -- if have been detected early, may have a way to stop it?
Some automatic transfer may have been used since some transfer amounts are very regular (i.e. same number in multiple transfers) -- this could be a sign of rug-pulling?

### clues from the twitter account?
it's all been deleted...

### chainalysis sanction list

